In [135]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import cPickle as pickle
import numpy as np

import nltk
import gensim
from unidecode import unidecode
import string

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import random 

from matplotlib import pyplot as plt
import seaborn as sns
% matplotlib inline

In [130]:
#all_menus.to_pickle('all_menus.p')
all_menus = pd.read_pickle('all_menus.p')

#full_menus = all_menus.groupby('name_and_address').agg({'tokens': 'sum'}).reset_index()
#full_menus.to_pickle('full_menus.p')
full_menus = pd.read_pickle('full_menus.p')

# pickle.dump(tokens, open( "old_tokens.p", "wb" ) )
# old_tokens was obtained by removing punctuation, tokenizing, lemmatizing, making lower case and removing duplicate
# words from items in menu
old_tokens = pickle.load(open( "old_tokens.p", "rb" ))

# pickle.dump(new_tokens, open( "new_tokens.p", "wb" ) )
# new_tokens are the tokens gotten from taking old_tokens and
# a) removing words that only appear once in corpus and
# b) removing single letters and c) removing numbers
new_tokens = pickle.load(open( "new_tokens.p", "rb" ))

# pickle.dump(tfidf_mat_menus, open( "tfidf_mat_menus.p", "wb" ) )
tfidf_mat_menus = pickle.load(open( "tfidf_mat_menus.p", "rb" ) )

#pickle.dump(tfidf_mat_rests, open( "tfidf_mat_rests.p", "wb" ) )
tfidf_mat_rests = pickle.load(open( "tfidf_mat_rests.p", "rb" ) )

# pickle.dump(unique_rest_id_list, open( "unique_rest_id_list.p", "wb" ) )
unique_rest_id_list = pickle.load(open( "unique_rest_id_list.p", "rb" ))

#pickle.dump(restaurant_name_list, open( "restaurant_name_list.p", "wb" ) )
restaurant_name_list = pickle.load(open( "restaurant_name_list.p", "rb" ) )

#pickle.dump(restaurant_address_list, open( "restaurant_address_list.p", "wb" ) )
restaurant_address_list = pickle.load(open( "restaurant_address_list.p", "rb" ) )

#pickle.dump(name_and_addr_list, open( "name_and_addr_list.p", "wb" ) )
name_and_addr_list = pickle.load(open( "name_and_addr_list.p", "rb" ) )

# pickle.dump(cos_sim_list, open( "cos_sim_list.p", "wb" ) )
cos_sim_list = pickle.load(open( "cos_sim_list.p", "rb" ) )

# pickle.dump(cost_sim_const_list, open( "cost_sim_const_list.p", "wb" ) )
cost_sim_const_list = pickle.load(open( "cost_sim_const_list.p", "rb" ) )

# pickle.dump(lift_list, open( "lift_list", "wb" ) )
lift_list = pickle.load(open( "lift_list", "rb" ) )

def tfIdfMatCreator(tokens):

    from sklearn.feature_extraction.text import TfidfVectorizer

    tfidf_model = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False)
    tfidf_mat = tfidf_model.fit_transform(tokens)
    return tfidf_mat

def getTopMatches(tfidf_mat,idx):
    
    # Compute the cosine similarity between the input idx and all the other vectors
    cosine_similarities = cosine_similarity(tfidf_mat[idx], tfidf_mat).flatten()

    # Organize the related food in descending order
    related_idcs = cosine_similarities.argsort()[::-1]
    
    # Organize the cosine similarities in descending order
    cosine_similarities_sorted = sorted(cosine_similarities, reverse=True)

    # Remove the user input
    user_selection = related_idcs[0]

    # The rest of them
    other_similar_items = related_idcs[1:]
    
    # Similarity metrics
    return user_selection,other_similar_items,cosine_similarities_sorted[1:]

In [153]:
jshack_idcs = all_menus[all_menus.restaurant_name=='J Shack'].index.tolist()

sinbad_idcs = all_menus[all_menus.restaurant_name=='Sinbad\'s'].index.tolist()

triptych_idcs = all_menus[all_menus.restaurant_name=='Triptych'].index.tolist()

In [154]:
idcs_to_drop = sinbad_idcs+triptych_idcs+jshack_idcs

In [155]:
idcs_to_keep = [x for x in range(all_menus.shape[0]) if x not in idcs_to_drop]

In [156]:
all_menus_new = all_menus.iloc[idcs_to_keep].reset_index()

In [157]:
new_tokens_new = [j for i,j in enumerate(new_tokens) if i not in idcs_to_drop]

In [158]:
all_menus_new.tokens = new_tokens_new

In [159]:
print all_menus.iloc[12431].item_description
print all_menus.iloc[12431].tokens

juicy all white breast of turkey with mayo, onion, lettuce and tomato
[u'turkey', u'tomato', u'onion', u'mayo', u'juicy', u'lettuce', u'breast', u'white']


In [162]:
drop_list = ['J Shack, Location Varies, San Francisco, CA, 94103',"Sinbad's, Pier 2 Embarcadero St, San Francisco, CA, 94111",'Triptych, 1155 Folsom St, San Francisco, CA, 94103']
name_and_addr_list_new = [x for x in name_and_addr_list if x not in drop_list]

drop_list = ['J Shack',"Sinbad's",'Triptych']
restaurant_name_list_new = [x for x in restaurant_name_list if x not in drop_list]

name_and_addr_list_split = [x.split(', ',1) for x in name_and_addr_list_new]

In [163]:
print len(restaurant_name_list_new)
print len(name_and_addr_list_new)
print len(name_and_addr_list_split)

1011
1011
1011


In [164]:
test_idx = -1
print restaurant_name_list_new[test_idx]
print name_and_addr_list_new[test_idx]
print name_and_addr_list_split[test_idx]

yoCup
yoCup, 685 Market St, San Francisco, CA, 94105
['yoCup', '685 Market St, San Francisco, CA, 94105']


In [165]:
tfidf_mat_menus_new = tfIdfMatCreator(new_tokens_new)

In [166]:
tfidf_mat_menus_new

<74867x9653 sparse matrix of type '<type 'numpy.float64'>'
	with 557907 stored elements in Compressed Sparse Row format>

# Pickle everything important

In [167]:
pickle.dump(name_and_addr_list_new, open( "name_and_addr_list_new.p", "wb" ) )
pickle.dump(name_and_addr_list_split, open( "name_and_addr_list_split.p", "wb" ) )
pickle.dump(tfidf_mat_menus_new, open( "tfidf_mat_menus_new.p", "wb" ) )
all_menus_new.to_pickle('all_menus_new.p')

In [191]:
all_menus_new = pd.read_pickle('all_menus_new.p')

In [192]:
all_menus_new[all_menus_new.restaurant_name=='Ti-couz - CLOSED']

,index,category_description,category_name,city,full_address,item_description,item_name,item_price,num_reviews,restaurant_category,restaurant_name,state,street_address,yelp_link,yelp_rating,zip,name_and_address,tokens
74778,75016,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",Plain savory crepe,Nature,$2.00,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[savory, plain, crepe, nature]"
74779,75017,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",ham savory crepe,Jambon,$4.50,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[savory, jambon, crepe, ham]"
74780,75018,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",butter savory crepe,Beurre,$2.50,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[butter, savory, beurre, crepe]"
74781,75019,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",flavored butter (basil or parsley or garlic or...,Beurre Parfume,$3.50,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[butter, beurre, crepe, parfume, parsley, savo..."
74782,75020,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",egg savory crepe,Oeuf,$3.50,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[savory, crepe, egg, oeuf]"
74783,75021,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",cheese savory crepe,Fromage,$4.50,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[savory, cheese, crepe, fromage]"
74784,75022,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",caramelized onions savory crepe,Oignon Caramalise,$5.00,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[crepe, onion, oignon, savory, caramelized]"
74785,75023,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",organic tomatoes (with basil butter) savory crepe,Tomate Organique,$5.25,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[tomato, butter, organic, tomate, savory, basi..."
74786,75024,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",ratatouille savory crepe,Ratatouille,$6.00,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[savory, crepe, ratatouille]"
74787,75025,can all be garnished with homemade creme fraic...,Krampouz Ble Noir - Savory Crepe,San Francisco,"3108 16th St, San Francisco, CA, 94103",sausage (with basil butter) savory crepe,Saucisse,$6.25,1148.0,c,Ti-couz - CLOSED,CA,3108 16th St,http://www.yelp.com/biz/ti-couz-san-francisco,4.0,94103,Ti-couz - CLOSED 3108 16th St,"[butter, sausage, crepe, saucisse, savory, basil]"


In [193]:
sav_crepe_list = all_menus_new[all_menus_new.category_name == 'Krampouz Ble Noir - Savory Crepe'].index.tolist()

for i in sav_crepe_list:
    all_menus_new.iloc[i,6] = all_menus_new.iloc[i,6] + ' savory crepe'

In [195]:
sweet_crepe_list = all_menus_new[all_menus_new.category_name == 'Krampouz Froment - Sweet Crepe'].index.tolist()

for i in sweet_crepe_list:
    all_menus_new.iloc[i,6] = all_menus_new.iloc[i,6] + ' sweet crepe'

In [197]:
all_menus_new.to_pickle('all_menus_new.p')